# **GOOGLE SHEETS LEADS DASHBOARD**

In [1]:
import pandas as pd
import numpy as np

import gspread

from gspread_pandas import Spread
from google.oauth2.service_account import Credentials

scopes = ['https://www.googleapis.com/auth/spreadsheets', 
          'https://www.googleapis.com/auth/drive']

credentials = Credentials.from_service_account_file('gspread_service_account.json', scopes = scopes)

gc = gspread.authorize(credentials)

## **1. DATA LOADING AND PREPROCESSING**

In [2]:
df = pd.read_csv('lead_report_dataset.csv', index_col = [0])

In [3]:
df.head()

,lead_id,product,is_referral,referal_cost,device,marketing_source,lead_cost,unqualified_lead,qualified_lead,meeting_appointment,meeting,sale
0,154236.0,Main Product,True,1000.0,NaN,NaN,0.000000,2021-07-01,2021-07-01,NaN,NaN,NaN
1,193988.0,Main Product,False,0.0,Mobile,Search Network,33.743385,2021-07-01,2021-07-03,NaN,NaN,NaN
2,101517.0,Main Product,False,0.0,Mobile,Paid Search,31.888973,2021-07-01,2021-07-01,2021-07-09,2021-07-10,2021-07-16
3,101634.0,Main Product,False,0.0,Mobile,Direct Traffic,0.000000,2021-07-01,NaN,NaN,NaN,NaN
4,165569.0,Main Product,False,0.0,Mobile,Search Network,39.631895,2021-07-01,2021-07-01,NaN,NaN,NaN


In [4]:
print('Number of observations:', len(df))

Number of observations: 8060


**— Converting date values to datetime format.**

In [5]:
for column in ['unqualified_lead', 'qualified_lead', 'meeting_appointment', 'meeting', 'sale']: 
    
    df[column] = pd.to_datetime(df[column])

In [6]:
def date_to_text_month(date_timestamp):
    
    date_text_list = []
    month = ''
    
    if pd.isnull(date_timestamp) == False: 
        
        date_text_list = str(date_timestamp).split('-')
        month = date_text_list[1] + '.' + date_text_list[0]
    
    return month

In [7]:
def date_to_text_half_month(date_timestamp):

    date_text_list = []
    half_month = ''
    
    if pd.isnull(date_timestamp) == False: 
        
        date_text_list = str(date_timestamp).split('-')
        
        if int(date_text_list[2].split(' ')[0]) <= 15: 
            
            half_month = '01.' + date_text_list[1] + '.' + date_text_list[0]
            
        if int(date_text_list[2].split(' ')[0]) > 15: 
            
            half_month = '16.' + date_text_list[1] + '.' + date_text_list[0]
    
    return half_month

In [8]:
for date_column in ['unqualified_lead', 'qualified_lead', 'meeting_appointment', 'meeting', 'sale']: 
    
    df[f'{date_column}_month'] = df[date_column].apply(date_to_text_month)
    df[f'{date_column}_half_month'] = df[date_column].apply(date_to_text_half_month)

**— Grouping data into 3 dataframes to further visualize general metrics and metrics based on device type and marketing source.**

In [9]:
df_analytics_general = pd.DataFrame(
    df.groupby(
        ['product', 'is_referral', 
         'unqualified_lead_month', 'unqualified_lead_half_month', 
         'qualified_lead_month', 'qualified_lead_half_month', 
         'meeting_appointment_month', 'meeting_appointment_half_month', 
         'meeting_month', 'meeting_half_month', 
         'sale_month', 'sale_half_month']
    ).agg(
        sum_lead_cost = pd.NamedAgg('lead_cost', 'sum'), 
        sum_referal_cost = pd.NamedAgg('referal_cost', 'sum'), 
        count_unqualified_lead = pd.NamedAgg('lead_id', 'count'), 
        count_qualified_lead = pd.NamedAgg('qualified_lead', 'count'), 
        count_meeting_appointment = pd.NamedAgg('meeting_appointment', 'count'), 
        count_meeting = pd.NamedAgg('meeting', 'count'), 
        count_sale = pd.NamedAgg('sale', 'count')
    )
).reset_index()

In [10]:
df_analytics_device = pd.DataFrame(
    df[df['is_referral'] == False].groupby(
        ['device', 'product', 
         'unqualified_lead_month', 'unqualified_lead_half_month', 
         'qualified_lead_month', 'qualified_lead_half_month', 
         'meeting_appointment_month', 'meeting_appointment_half_month', 
         'meeting_month', 'meeting_half_month', 
         'sale_month', 'sale_half_month']
    ).agg(
        sum_lead_cost = pd.NamedAgg('lead_cost', 'sum'), 
        sum_referal_cost = pd.NamedAgg('referal_cost', 'sum'), 
        count_unqualified_lead = pd.NamedAgg('lead_id', 'count'), 
        count_qualified_lead = pd.NamedAgg('qualified_lead', 'count'), 
        count_meeting_appointment = pd.NamedAgg('meeting_appointment', 'count'), 
        count_meeting = pd.NamedAgg('meeting', 'count'), 
        count_sale = pd.NamedAgg('sale', 'count')
    )
).reset_index()

In [11]:
df_analytics_marketing_source = pd.DataFrame(
    df[df['is_referral'] == False].groupby(
        ['marketing_source', 'product', 
         'unqualified_lead_month', 'unqualified_lead_half_month', 
         'qualified_lead_month', 'qualified_lead_half_month', 
         'meeting_appointment_month', 'meeting_appointment_half_month', 
         'meeting_month', 'meeting_half_month', 
         'sale_month', 'sale_half_month']
    ).agg(
        sum_lead_cost = pd.NamedAgg('lead_cost', 'sum'), 
        sum_referal_cost = pd.NamedAgg('referal_cost', 'sum'), 
        count_unqualified_lead = pd.NamedAgg('lead_id', 'count'), 
        count_qualified_lead = pd.NamedAgg('qualified_lead', 'count'), 
        count_meeting_appointment = pd.NamedAgg('meeting_appointment', 'count'), 
        count_meeting = pd.NamedAgg('meeting', 'count'), 
        count_sale = pd.NamedAgg('sale', 'count')
    )
).reset_index()

**— Formating data to confirm Google Sheets requirements.**

In [12]:
df_to_unload_general = df_analytics_general.copy()

for column in ['unqualified_lead_half_month', 'qualified_lead_half_month', 
               'meeting_appointment_half_month', 'meeting_half_month', 'sale_half_month']: 
    
    df_to_unload_general.loc[df_to_unload_general[column].str[:2] == '01', f'{column}_perdiod'] = '15 days'
    df_to_unload_general.loc[df_to_unload_general[column].str[:2] == '16', f'{column}_perdiod'] = '30 days'

for column in ['sum_lead_cost', 'sum_referal_cost', 'count_unqualified_lead', 'count_qualified_lead', 
               'count_meeting_appointment', 'count_meeting', 'count_sale']: 
    
    df_to_unload_general[column] = df_to_unload_general[column].astype(str).apply(lambda x: x.replace('.', ','))
    df_to_unload_general.loc[df_to_unload_general[column] == 'nan', column] = ''

In [13]:
df_to_unload_device = df_analytics_device.copy()

for column in ['unqualified_lead_half_month', 'qualified_lead_half_month', 
               'meeting_appointment_half_month', 'meeting_half_month', 'sale_half_month']: 
    
    df_to_unload_device.loc[df_to_unload_device[column].str[:2] == '01', f'{column}_perdiod'] = '15 days'
    df_to_unload_device.loc[df_to_unload_device[column].str[:2] == '16', f'{column}_perdiod'] = '30 days'

for column in ['sum_lead_cost', 'sum_referal_cost', 'count_unqualified_lead', 'count_qualified_lead', 
               'count_meeting_appointment', 'count_meeting', 'count_sale']: 
    
    df_to_unload_device[column] = df_to_unload_device[column].astype(str).apply(lambda x: x.replace('.', ','))
    df_to_unload_device.loc[df_to_unload_device[column] == 'nan', column] = ''

In [14]:
df_to_unload_marketing_source = df_analytics_marketing_source.copy()

for column in ['unqualified_lead_half_month', 'qualified_lead_half_month', 
               'meeting_appointment_half_month', 'meeting_half_month', 'sale_half_month']: 
    
    df_to_unload_marketing_source.loc[df_to_unload_marketing_source[column].str[:2] == '01', f'{column}_perdiod'] = '15 days'
    df_to_unload_marketing_source.loc[df_to_unload_marketing_source[column].str[:2] == '16', f'{column}_perdiod'] = '30 days'

for column in ['sum_lead_cost', 'sum_referal_cost', 'count_unqualified_lead', 'count_qualified_lead', 
               'count_meeting_appointment', 'count_meeting', 'count_sale']: 
    
    df_to_unload_marketing_source[column] = df_to_unload_marketing_source[column].astype(str).apply(lambda x: x.replace('.', ','))
    df_to_unload_marketing_source.loc[df_to_unload_marketing_source[column] == 'nan', column] = ''

## **2. DATA UNLOADING**

In [15]:
spread = Spread('1LolOZsqOVswfLzbsgCIa646TwI4Qe7XqN-X0QkweaxM', creds = credentials)

In [16]:
spread.df_to_sheet(df_to_unload_general, sheet = 'Data | General', index = False)
spread.df_to_sheet(df_to_unload_device, sheet = 'Data | Devices', index = False)
spread.df_to_sheet(df_to_unload_marketing_source, sheet = 'Data | Marketing Sources', index = False)